## Implement unpadded message recovery oracle

Nate Lawson says we should stop calling it "RSA padding" and start calling it "RSA armoring". Here's why.

Imagine a web application, again with the Javascript encryption, taking RSA-encrypted messages which (again: Javascript) aren't padded before encryption at all.

You can submit an arbitrary RSA blob and the server will return plaintext. But you can't submit the same message twice: let's say the server keeps hashes of previous messages for some liveness interval, and that the message has an embedded timestamp:

```json
{
  time: 1356304276,
  social: '555-55-5555',
}
```

You'd like to capture other people's messages and use the server to decrypt them. But when you try, the server takes the hash of the ciphertext and uses it to reject the request. Any bit you flip in the ciphertext irrevocably scrambles the decryption.

This turns out to be trivially breakable: 

* Capture the ciphertext C
* Let N and E be the public modulus and exponent respectively
* Let S be a random number > 1 mod N. Doesn't matter what.
* Now:

```
C' = ((S**E mod N) C) mod N
```

* Submit C', which appears totally different from C, to the server, recovering P', which appears totally different from P
Now:

```
          P'
    P = -----  mod N
          S
```

 Oops!

Implement that attack.
### Careful about division in cyclic groups.
Remember: you don't simply divide mod N; you multiply by the multiplicative inverse mod N. So you'll need a modinv() function.




### Unpadded Message Oracle
Configure the server's RSA keypair and textbook RSA decrypt function.

In [1]:
from Crypto.Util.number import getPrime

In [2]:
NUM_BITS = 1024
p = getPrime(NUM_BITS)
q = getPrime(NUM_BITS)

In [3]:
n = p * q
n.bit_length()

2047

In [5]:
phi = (p - 1) * (q - 1) # Euler's totient.

In [7]:
d = pow(e, -1, phi)
d

9999084168040618735634861551028646714742419824776844939618046517930489960096612274105788204465626306989470696783698343279280014861995265035569539280891631064297939780160124628869695269343557975135982891073329303818009473603698633092090004042698768900183665105420632261310775144887374464668624168201333009266899950812475295837304653006366264235873345873988264006742306399022556182903044439835865635144643674405954517323848976805487216372176489900871366907034146005163955964589454433788255139460061307379840363093156441943600475313978033317975884382581433172679113666070300725777779503999992424431816057691268724076147

In [9]:
assert e * d % phi == 1

In [5]:
N = 14998626252060928103452292326542970072113629737165267409427069776895734940144918411158682306698439460484206045175547514918920022292992897553354308921337446596446909670240186943304542904015336962703974336609993955727014210405547949638135006064048153350275497658130948391966162717331061697002936252301999513900597824741726854402436970305499076189535485327631912218072518428911792985143753609764640004919486259062931216650284588585901673444541101407504191882322230182874555079901080636451642823862690883410812084807478665973587954142541349197747345526288869524033493400650480652831708745653869356203205959909973122017029
e = 3 # public exponent.

In [2]:
from Crypto.Util.number import bytes_to_long, long_to_bytes

In [3]:
m = b"retreat"

In [6]:
C = pow(bytes_to_long(m), e, N)

In [7]:
def decrypt(c):
    N = 14998626252060928103452292326542970072113629737165267409427069776895734940144918411158682306698439460484206045175547514918920022292992897553354308921337446596446909670240186943304542904015336962703974336609993955727014210405547949638135006064048153350275497658130948391966162717331061697002936252301999513900597824741726854402436970305499076189535485327631912218072518428911792985143753609764640004919486259062931216650284588585901673444541101407504191882322230182874555079901080636451642823862690883410812084807478665973587954142541349197747345526288869524033493400650480652831708745653869356203205959909973122017029
    d = 9999084168040618735634861551028646714742419824776844939618046517930489960096612274105788204465626306989470696783698343279280014861995265035569539280891631064297939780160124628869695269343557975135982891073329303818009473603698633092090004042698768900183665105420632261310775144887374464668624168201333009266899950812475295837304653006366264235873345873988264006742306399022556182903044439835865635144643674405954517323848976805487216372176489900871366907034146005163955964589454433788255139460061307379840363093156441943600475313978033317975884382581433172679113666070300725777779503999992424431816057691268724076147
    return long_to_bytes(pow(c, d, N))

Test the "oracle".

In [8]:
decrypt(C)

b'retreat'